## HelloMember – Multiple application calls in one block
#### 07.02a Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-09

* Multiple application calls within a single block

## Setup
See notebook 04.1, loading `algo_util.py`, the five accounts and the Purestake credentials
* Consider hiding this code

In [ ]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json
import base64
import datetime

In [ ]:
from pyteal import *

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

In [ ]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

## HelloMember with multiple opt-ins in one block
* **Exactly the same program** as in `07.02_WSC`
* User interactions are **different**
    * What happens if multiple users opt-in at the same time?
    * Is the total number of members and the individual membership number calculated correctly?

*Spoiler alert:* yes!

#### Step 1: Define Approval program

In [ ]:
handle_creation = Seq(
    [
        # Initialize a GLOBAL variable
        App.globalPut(Bytes("Note"),       Bytes("Hello world!")),
        App.globalPut(Bytes("Members"),    Int(0)),                    # <---- initialize: currently we have zero members
        Return(Int(1))                                                 # Return "OK"
    ]
)

handle_optin = Seq(
    [
        App.globalPut(                                  # GLOBAL: increase number of Members
            Bytes("Members"),                           # key
            App.globalGet(Bytes("Members")) + Int(1)    # value: add 1 to current number of members
            ),
        App.localPut(                                   # LOCAL: membership number
            Int(0),                                     # which user? current one!
            Bytes("MembershipNo"),                      # key
            App.globalGet(Bytes("Members"))             # value: current members (already increased by 1)
        ),
        Return(Int(1))                                  # Return "OK"
    ]
)


handle_closeout = Return( Int(1) )                      # Not doing anything, returning "OK"  

handle_updateapp = Return( Int(0) )                     # Always FALSE ... updating not allowed

handle_deleteapp = Return(
    Txn.sender() == Global.creator_address()            # only TRUE if delete request is made by app creator
)

handle_noop  = Return ( Int(1) )                        # Not doing anything, returning "OK"           

hellomember_approval_pyteal = Cond(
    [Txn.application_id() == Int(0),                       handle_creation],
    [Txn.on_completion()  == OnComplete.OptIn,             handle_optin],
    [Txn.on_completion()  == OnComplete.CloseOut,          handle_closeout],
    [Txn.on_completion()  == OnComplete.UpdateApplication, handle_updateapp],
    [Txn.on_completion()  == OnComplete.DeleteApplication, handle_deleteapp],
    [Txn.on_completion()  == OnComplete.NoOp,              handle_noop],
)

# Compile PyTEAL -> TEAL
hellomember_approval_teal = compileTeal(hellomember_approval_pyteal, mode=Mode.Application, version=5)
#print(hellomember_approval_teal)

#### Step 1b: Define Clear State program

In [ ]:
hellomember_clear_pyteal =  Return(Int(1))    # not doing anything

# Compile PyTEAL -> TEAL
hellomember_clear_teal = compileTeal(hellomember_clear_pyteal,mode=Mode.Application, version=3)
#print(hellomember_clear_teal)

#### Compile TEAL -> Bytecode

In [ ]:
hellomember_approval_b64 = algod_client.compile(hellomember_approval_teal)
Hellomember_Approval =  base64.b64decode(hellomember_approval_b64['result'])

hellomember_clear_b64 = algod_client.compile(hellomember_clear_teal)
Hellomember_Clear =  base64.b64decode(hellomember_clear_b64['result'])

## Deploy Smart Contract

##### Bob deploys the smart contract

In [ ]:
# Step 1: Prepare the transaction
sp = algod_client.suggested_params()

# Reserve space
global_ints = 1    # for "Members"
global_bytes = 1   # for "Note"
hellomember_global_schema = transaction.StateSchema(global_ints, global_bytes)

local_ints = 1     # for "MembershipNo"
local_bytes = 0    # No local text var
hellomember_local_schema = transaction.StateSchema(local_ints, local_bytes)

txn = transaction.ApplicationCreateTxn(
      sender = Bob['public'],                    # <-- sender public
      sp = sp,                                   # <-- sp
      on_complete = 0,                           # <-- when finished do nothing
      approval_program = Hellomember_Approval,   # <-- approval program 
      clear_program = Hellomember_Clear,         # <-- clear program 
      global_schema = hellomember_global_schema, # <-- reserve global space 
      local_schema = hellomember_local_schema    # <-- reserve local space
    )

# Step 2: sign transaction
stxn = txn.sign(Bob['private'])

# Step 3: send
txid=algod_client.send_transactions([stxn])

# Step 4: wait for ...
txinfo = wait_for_confirmation(algod_client, txid)

In [ ]:
app_id = txinfo["application-index"]
print("Created new app-id:", app_id)

## The Smart Contract is now deployed

In [ ]:
format_state(read_global_state(algod_client,app_id))

## Using the Smart Contract: Opt-in
* What happens if multiple users opt-in at the same time?
    * In rare cases, it may happen that not all four transactions are processed in the same block
    * The conclusions that we can draw are still valid

In [ ]:
# create a list of transactions
#users = [Alice, Bob, Charlie, Dina]
users = [Dina, Alice, Bob, Charlie]

sp = algod_client.suggested_params()

txids=[]
for user in users:
    # Step 1: prepare transaction
    txn = transaction.ApplicationOptInTxn(user['public'], sp, app_id)
    # Step 2: sign transaction
    stxn = txn.sign(user['private'])
    # Step 3: send
    txids.append( algod_client.send_transactions( [stxn] ) )

In [ ]:
# Step 4: await confirmation
for txid in txids:
    txinfo = wait_for_confirmation(algod_client, txid)

## Inspect the global state and the local states of the users

In [ ]:
format_state(read_global_state(algod_client,app_id))

In [ ]:
read_local_state(algod_client,Alice['public'],app_id)

In [ ]:
read_local_state(algod_client,Bob['public'],app_id)

In [ ]:
read_local_state(algod_client,Charlie['public'],app_id)

In [ ]:
read_local_state(algod_client,Dina['public'],app_id)

## Calling the Smart contract
* Does not really make sense here ...

## Deleting the app
* Rather important, because an address can only create **10 apps**
* App can be deleted by creator

In [ ]:
creator = Bob

# Step 1: Prepare transaction
sp = algod_client.suggested_params()
txn = transaction.ApplicationDeleteTxn(creator['public'], sp, app_id)

# Step 2: sign
stxn = txn.sign(creator['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step4 : wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

# display results
print("Deleted app-id:", txinfo["txn"]["txn"]["apid"])

## Conclusion
* Algorand Smart Contracts can handle multiple user interactions (opt-in, application call) within one block
* Even within a block, user interactions are processed in the order of their arrival (i.e. when they have been posted to the blockchain)

## Exercise
* Delete the app, if you have not yet done so
* Go to the section **Using the Smart Contract: Opt-in**
* Change the order of users in the code line

```{python}
users = [Alice, Bob, Charlie, Dina]
```

* Re-run the entire notebook and inspect the (new) membership numbers